In [1]:
import os
import cv2
import numpy as np
from pathlib import Path  # For handling file paths

In [2]:
original_dataset_path = r"D:\MED_LEAF_ID\data\dataset\Medicinal Leaf dataset"
augmented_dataset_path = r"D:\MED_LEAF_ID\data\augmented"
preprocessed_original_path = r"D:\MED_LEAF_ID\data\cnn\original"
preprocessed_augmented_path = r"D:\MED_LEAF_ID\data\cnn\augmented"


In [3]:
def create_directory_structure(dataset_path, preprocessed_path):
    if not os.path.exists(preprocessed_path):
        os.makedirs(preprocessed_path)
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            os.makedirs(os.path.join(preprocessed_path, class_folder), exist_ok=True)


In [4]:
def preprocess_image_for_cnn(image_path):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None
    
    # Resize to a consistent size (adjust as needed)
    resized_image = cv2.resize(image, (224, 224))
    
    # Normalize the image (convert to range [0, 1])
    normalized_image = resized_image.astype('float32') / 255.0
    
    return normalized_image

In [5]:
def preprocess_and_save_images(dataset_path, preprocessed_path):
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            save_path = os.path.join(preprocessed_path, class_folder)
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                processed_image = preprocess_image_for_cnn(image_path)
                if processed_image is not None:
                    # Save preprocessed image (normalized)
                    cv2.imwrite(os.path.join(save_path, image_file), (processed_image * 255).astype(np.uint8))
                else:
                    print(f"Skipping image: {image_file}")


In [6]:
if __name__ == "__main__":
    print("Creating directory structure for original dataset preprocessing...")
    create_directory_structure(original_dataset_path, preprocessed_original_path)
    print("Creating directory structure for augmented dataset preprocessing...")
    create_directory_structure(augmented_dataset_path, preprocessed_augmented_path)
    
    print("Preprocessing original images and saving to the new directory...")
    preprocess_and_save_images(original_dataset_path, preprocessed_original_path)
    
    print("Preprocessing augmented images and saving to the new directory...")
    preprocess_and_save_images(augmented_dataset_path, preprocessed_augmented_path)
    
    print("Preprocessing complete!")

Creating directory structure for original dataset preprocessing...
Creating directory structure for augmented dataset preprocessing...
Preprocessing original images and saving to the new directory...
Preprocessing augmented images and saving to the new directory...
Preprocessing complete!
